# HW4 Part 1: Figure Out a Metabolic Pathway for Synthesis of a Chemical of Your Choice and Balance It

In class, we have discussed a theory that let’s us calculate the efficiency of electromicrobial production (EMP). Calculating this efficiency involves figuring out how many electrons the cell takes up to make a single molecule of the target product (e.g., butanol). The number of electrons is directly related to the number of NADH, ATP, and Ferredoxin that the cell needs for biochemical synthesis of the target.

In the next two homeworks (HW7 and HW8), you will team up and choose a molecule to synthesize, work together to figure out the balance of chemicals in these metabolic synthesis pathways, and finally calculate the efficiency of production (in HW8).

For this homework, in addition to submitting this Jupyter notebook, you'll also need to submit:

* A table (Excel file converted to pdf) of the reactions along with references.
* A very short essay on why you chose your molecule.
* Don't forget, I want a submission from everyone, even though you are working in teams (individual submissions from each team member can be identical though).


In [ ]:
#Setup: fetch public utils via raw.githubusercontent
import os, sys, pathlib, requests, importlib

UTILS_REF = "main"
BASE = f"https://raw.githubusercontent.com/BEE3600-2025/BEE-3600-Assets/{UTILS_REF}/utils"
DEST = pathlib.Path("/content/bee3600_utils/utils"); DEST.mkdir(parents=True, exist_ok=True)

files = ["__init__.py","balanceUtils.py","specutils12.py","vectorOutput4.py"]
for f in files:
    url = f"{BASE}/{f}"
    r = requests.get(url, timeout=30); r.raise_for_status()
    (DEST/f).write_bytes(r.content)
    print("✔︎ fetched", f)

ROOT = str(DEST.parent)
if ROOT not in sys.path: sys.path.insert(0, ROOT)

for m in ("utils.balanceUtils","utils.specutils12","utils.vectorOutput4"):
    importlib.import_module(m)
print("utils ready")


## Initialization

In [ ]:
import re
import os.path
import pdb

from numpy import unique, int8, array

from utils.balanceUtils import SolveFluxBalanceEquation, ConvertIndexedSMatrix, ImportIOStatus, \
ImportReactionFile, GenerateIndexedSMatrixT, PrintStoichiometry, GenerateMergedIOStatusList, \
Get_IO_Status_for_Compound_List, ExportUniqueCompoundsWithIOStatus, GenerateIOStatusList, ParseReactionList

from utils.vectorOutput4 import generateOutputMatrixWithHeaders, writeOutputMatrix, Write_SMatrix
from utils.specutils12 import ensure_dir

## Reaction List

Use the KEGG and BRENDA databases to figure out the reactions needed to synthesize a chemical of your choice.

In [ ]:
reactionList = \
['2*CO2 + 7*ATP + 4*NADH -> 1*Acetyl-CoA', \
 '3*CO2 + 7*ATP + 5*NADH -> 1*Pyruvate', \
 '16*ATP + 8*Fd1red + N2 → 2*NH3 + 1*H2', \
'2*Fd1red + Hplus + NADplus -> 2*Fd1 + NADH', \
'2*Acetyl-CoA -> CoA + Acetoacetyl-CoA', \
'Acetoacetyl-CoA + NADPH + Hplus -> Hydroxybutanoyl-CoA + NADPplus', \
'Hydroxybutanoyl-CoA -> Crotonoyl-CoA + H2O', \
'Crotonoyl-CoA + NADH + Hplus -> Butanoyl-CoA + NADplus', \
'Butanoyl-CoA + NADH + Hplus -> Butanal + CoA + NADplus', \
'Butanal + NADH + Hplus -> 1-Butanol + NADplus', \
'NADH + NADPplus -> NADPH + NADplus', \
'AMP + ATP -> 2*ADP' \
]

In [ ]:
reactantsToGet = ['ATP', 'NADH', 'Fdred', 'Sulfate', 'N2', 'CO2', 'HCO3-', 'HCO2-', \
'CO','Fd1red']

## Prepare Stoichiometric Matrix

Figure out the unique compounds, reactions, and calculate a stoichiomietric matrix.

In [ ]:
uniqueCompounds, reactions, sMatrixTKeyIndexed = ParseReactionList(reactionList, reactionArrow='->')

In [ ]:
uniqueCompoundsIOStatusProposed = GenerateIOStatusList(uniqueCompounds)
uniqueCompoundsIOStatusProposed

## Edit This

Make your best guess about the input/output status of each compound. Input/Output, Input, and Output are all the same from the point of view of the code. The crucial difference is between Target (one allowed); Intermediate; and Input|Output.

In [ ]:
uniqueCompoundsIOStatusEdit = \
[['1-Butanol', 'Target'],
 ['13BPG', 'Intermediate'],
 ['2PG', 'Intermediate'],
 ['3PG', 'Intermediate'],
 ['ADP', 'Input/Output'],
 ['AMP', 'Input/Output'],
 ['ATP', 'Input'],
 ['Acetoacetyl-CoA', 'Intermediate'],
 ['Acetyl-CoA', 'Intermediate'],
 ['Butanal', 'Intermediate'],
 ['Butanoyl-CoA', 'Intermediate'],
 ['CO2', 'Input/Output'],
 ['CoA', 'Input/Output'],
 ['Crotonoyl-CoA', 'Intermediate'],
 ['E4P', 'Intermediate'],
 ['F16BP', 'Intermediate'],
 ['F6P', 'Intermediate'],
 ['Fd1', 'Intermediate'],
 ['Fd1red', 'Intermediate'],
 ['G3P', 'Intermediate'],
 ['GlyceronePhosph', 'Intermediate'],
 ['H2O', 'Input/Output'],
 ['Hplus', 'Input/Output'],
 ['Hydroxybutanoyl-CoA', 'Intermediate'],
 ['NADH', 'Input'],
 ['NADPH', 'Intermediate'],
 ['NADPplus', 'Intermediate'],
 ['NADplus', 'Input/Output'],
 ['P', 'Input/Output'],
 ['PEP', 'Intermediate'],
 ['Pyruvate', 'Intermediate'],
 ['Ribose5P', 'Intermediate'],
 ['Ribulose15BP', 'Intermediate'],
 ['Ribulose5P', 'Intermediate'],
 ['SH17BP', 'Intermediate'],
 ['SH7P', 'Intermediate'],
 ['X5P', 'Intermediate']]

i = 0
ioStatusEdit = []
while i < len(uniqueCompoundsIOStatusEdit):
    ioStatusEdit.append(uniqueCompoundsIOStatusEdit[i][1])
    i += 1

## Balance the Stoichiometric Matrix

In [ ]:
sMatrixT = ConvertIndexedSMatrix(sMatrixTKeyIndexed, uniqueCompounds)
sMatrix = sMatrixT.transpose()


[fVectorOpt, cDotVectorOpt, cDotVectorOptNorm, result] = \
SolveFluxBalanceEquation(sMatrix, reactions, uniqueCompounds, ioStatusEdit)

In [ ]:
[fVectorOpt, cDotVectorOpt, cDotVectorOptNorm, result]

In [ ]:
i = 0
printEverything = False
while i < len(uniqueCompounds):
        ioStatus = ioStatusEdit[i]
        compound = uniqueCompounds[i]
        if printEverything == False and (compound in reactantsToGet) or ioStatus == 'Target':
            print(uniqueCompounds[i] + ':\t' + ioStatus + '\t' + "%.1f" % cDotVectorOptNorm[i])
        elif printEverything == True:
            print(uniqueCompounds[i] + ':\t' + ioStatus + '\t' + "%.1f" % cDotVectorOptNorm[i])
        i += 1